# 2. OpenAI の チャット API の基礎


## 2.3. 入出力の長さの制限や料金に影響する「トークン」


### トークン


In [2]:
# tiktoken のインストール（Python 3.12以下で実行してください）
# Python 3.13を使用している場合は、このセルをスキップしてください
# !pip install tiktoken==0.7.0  # setup.shで事前インストール済み

In [1]:
import tiktoken

text = "ChatGPT"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
for token in tokens:
    print(encoding.decode([token]))

Chat
GPT


### Tokenizer と tiktoken の紹介


In [2]:
import tiktoken

text = "LLMを使ってクールなものを作るのは簡単だが、プロダクションで使えるものを作るのは非常に難しい。"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
print(len(tokens))

37


## 2.4. Chat Completions API を試す環境の準備


### OpenAI の API キーの準備


In [43]:
import os
from dotenv import load_dotenv

# .envファイルのパスを明示的に指定
# このノートブックが実行されているディレクトリに関わらず、
# プロジェクトルートのrag_ai_agent_bookディレクトリにある.envファイルを指定
current_working_directory = os.getcwd()
# os.getcwd()が/Users/kenichi/Projects/を返すので、そこからrag_ai_agent_book/.envへのパスを構築
dotenv_path = os.path.join(current_working_directory, 'rag_ai_agent_book', '.env')

print(f"現在の作業ディレクトリ: {current_working_directory}")
print(f"参照する.envパス: {dotenv_path}")

load_dotenv(dotenv_path=dotenv_path)

# 環境変数がロードされたか確認し、必要であればos.environに設定
api_key = os.getenv("OPENAI_API_KEY")

if api_key is None:
    print("OPENAI_API_KEY 環境変数が設定されていません。")
    print("rag_ai_agent_book/ディレクトリに.envファイルを作成し、OpenAI APIキーを設定してください。")
    print("例: OPENAI_API_KEY=\"sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx\"")
else:
    os.environ["OPENAI_API_KEY"] = api_key # 他のライブラリがos.environから直接読み込むことを期待する場合のために設定
    print("OPENAI_API_KEY が正常に設定されました。")


現在の作業ディレクトリ: /Users/kenichi/Projects
参照する.envパス: /Users/kenichi/Projects/rag_ai_agent_book/.env
OPENAI_API_KEY が正常に設定されました。


## 2.5. Chat Completions API のハンズオン


### OpenAI のライブラリ


#### 【注意】既知のエラーについて

openai パッケージが依存する httpx のアップデートにより、`openai==1.40.6` を使用する箇所で `TypeError: Client.__init__() got an unexpected keyword argument 'proxies'` というエラーが発生するようになりました。

このエラーは、`!pip install httpx==0.27.2` のように、httpx の特定バージョンをインストールすることで回避することができます。

なお、Google Colab で一度上記のエラーに遭遇したあとで `!pip install httpx==0.27.2` のようにパッケージをインストールし直した場合、以下のどちらかの操作を実施する必要があります。

- Google Colab の「ランタイム」から「セッションを再起動する」を実行する
- 「ランタイムを接続解除して削除」を実行してパッケージのインストールからやり直す


In [ ]:
# !pip install openai==1.40.6 httpx==0.27.2  # setup.shで事前インストール済み

### Chat Completions API の呼び出し


In [44]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"},
    ],
)
print(response.to_json(indent=4))

{
    "id": "chatcmpl-BqJJ00f29LJocghFnUjTrhvesA12E",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "こんにちは、ジョンさん！お会いできて嬉しいです。何かお手伝いできることがありますか？",
                "refusal": null,
                "role": "assistant",
                "annotations": []
            }
        }
    ],
    "created": 1751806966,
    "model": "gpt-4o-mini-2024-07-18",
    "object": "chat.completion",
    "service_tier": "default",
    "system_fingerprint": "fp_34a54ae93c",
    "usage": {
        "completion_tokens": 27,
        "prompt_tokens": 25,
        "total_tokens": 52,
        "prompt_tokens_details": {
            "cached_tokens": 0,
            "audio_tokens": 0
        },
        "completion_tokens_details": {
            "reasoning_tokens": 0,
            "audio_tokens": 0,
            "accepted_prediction_tokens": 0,
            "rejected_prediction_tokens": 0
        }
    }

### 会話履歴を踏まえた応答を得る


In [6]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私は西岡と言います！"},
        {"role": "assistant", "content": "こんにちは、西岡さん！お会いできて嬉しいです。今日はどんなことをお話ししましょうか？"},
        {"role": "user", "content": "私の名前が分かりますか？"},
    ],
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-Bq8XX1O4VIF7dQqduHTRGDhtlp2vQ",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "はい、あなたの名前は西岡さんです。何かお手伝いできることがあれば教えてください！",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  "created": 1751765583,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_34a54ae93c",
  "usage": {
    "completion_tokens": 27,
    "prompt_tokens": 68,
    "total_tokens": 95,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}


### ストリーミングで応答を得る


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私は西岡と言います！"},
    ],
    stream=True,
)

for chunk in response:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end="", flush=True)

こんにちは、西岡さん！お会いできて嬉しいです。今日はどのようなことについてお話ししましょうか？

### JSON モード


In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": '人物一覧を次のJSON形式で出力してください。\n{"people": ["aaa", "bbb"]}',
        },
        {
            "role": "user",
            "content": "昔々あるところにおじいさんとおばあさんがいました",
        },
    ],
    response_format={"type": "json_object"},
)
print(response.choices[0].message.content)

{"people": ["おじいさん", "おばあさん"]}


### Vision（画像入力）


In [46]:
from openai import OpenAI

client = OpenAI()

image_url = "https://raw.githubusercontent.com/yoshidashingo/langchain-book/main/assets/cover.jpg"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "画像を説明してください。"},
                {"type": "image_url", "image_url": {"url": image_url}},
            ],
        }
    ],
)

print(response.choices[0].message.content)

この画像は本の表紙を示しています。タイトルは「ChatGPT/LangChainによるチャットシステム構築【実践】入門」で、著者名が記載されています。表紙にはカラフルな鳥のイラストが描かれており、背景は明るい色合いです。また、「ChatGPT」という文字が大きく表示されており、内容に関連した情報が書かれています。本書は、AIモデルやシステム構築に関する知識や実践的なガイドを提供しているようです。


### （コラム）Completions API


In [47]:
from openai import OpenAI

client = OpenAI()

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt="こんにちは！私はジョンと言います！",
)
print(response.to_json(indent=2))

{
  "id": "cmpl-BqJPaLaClCSMOL77LtJcsKK6pVC7Y",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "');\n        followUp(event, inputText.text);\n      }\n      break;\n "
    }
  ],
  "created": 1751807374,
  "model": "gpt-3.5-turbo-instruct:20230824-v2",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 11,
    "total_tokens": 27
  }
}


## 2.6. Function calling


### Function calling のサンプルコード


In [48]:
import json

def get_current_weather(location, unit="fahrenheit"):
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": unit}
        )
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [49]:
# Function callingで使用するツールの定義
# OpenAIのFunction calling機能では、JSON Schema形式で関数の仕様を定義する
tools = [
    {
        "type": "function",  # ツールの種類を指定（function calling）
        "function": {
            "name": "get_current_weather",  # 呼び出したい関数名
            "description": "指定された場所の現在の天気を取得する",  # 関数の説明（LLMが関数を理解するために使用）
            "parameters": {
                "type": "object",  # パラメータの型（JSON Schema形式）
                "properties": {
                    "location": {
                        "type": "string",  # パラメータの型
                        "description": "都市と州、例：San Francisco, CA",  # パラメータの説明
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"]  # 選択可能な値の制限
                    },
                },
                "required": ["location"],  # 必須パラメータの指定
            },
        },
    }
]

In [50]:
from openai import OpenAI

client = OpenAI()

messages = [
    {"role": "user", "content": "東京の天気はどうですか？"},
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)
print(response.to_json(indent=4))

{
    "id": "chatcmpl-BqJPawLNehB5DSD5HLsl3PHWYho32",
    "choices": [
        {
            "finish_reason": "tool_calls",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": null,
                "refusal": null,
                "role": "assistant",
                "tool_calls": [
                    {
                        "id": "call_aaafNN2zCZTriNApCR2kOU2V",
                        "function": {
                            "arguments": "{\"location\":\"Tokyo, Japan\"}",
                            "name": "get_current_weather"
                        },
                        "type": "function"
                    }
                ],
                "annotations": []
            }
        }
    ],
    "created": 1751807374,
    "model": "gpt-4o-2024-08-06",
    "object": "chat.completion",
    "service_tier": "default",
    "system_fingerprint": "fp_07871e2ad8",
    "usage": {
        "completion_tokens": 17,
        "prom

In [51]:
response_message = response.choices[0].message
messages.append(response_message.to_dict())
print(messages)

[{'role': 'user', 'content': '東京の天気はどうですか？'}, {'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_aaafNN2zCZTriNApCR2kOU2V', 'function': {'arguments': '{"location":"Tokyo, Japan"}', 'name': 'get_current_weather'}, 'type': 'function'}], 'annotations': []}]


In [52]:
available_functions = {
    "get_current_weather": get_current_weather,
}

# 使いたい関数は複数あるかもしれないのでループ
for tool_call in response_message.tool_calls:
    # 関数を実行
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
    print(function_response)

    # 関数の実行結果を会話履歴としてmessagesに追加
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

{"location": "Tokyo", "temperature": "10", "unit": null}


In [53]:
print(json.dumps(messages, ensure_ascii=False, indent=2))

[
  {
    "role": "user",
    "content": "東京の天気はどうですか？"
  },
  {
    "content": null,
    "refusal": null,
    "role": "assistant",
    "tool_calls": [
      {
        "id": "call_aaafNN2zCZTriNApCR2kOU2V",
        "function": {
          "arguments": "{\"location\":\"Tokyo, Japan\"}",
          "name": "get_current_weather"
        },
        "type": "function"
      }
    ],
    "annotations": []
  },
  {
    "tool_call_id": "call_aaafNN2zCZTriNApCR2kOU2V",
    "role": "tool",
    "name": "get_current_weather",
    "content": "{\"location\": \"Tokyo\", \"temperature\": \"10\", \"unit\": null}"
  }
]


In [54]:
second_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)
print(second_response.to_json(indent=2))

{
  "id": "chatcmpl-BqJPfx7lnA1zJ04PV4klYIXdr21mm",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "東京の現在の気温は約10度です。具体的な天気の状態（晴れ、雨、曇りなど）はわかりませんが、全般的にこの時期は涼しいので、暖かい服装がおすすめです。",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  "created": 1751807379,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_a288987b44",
  "usage": {
    "completion_tokens": 58,
    "prompt_tokens": 59,
    "total_tokens": 117,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}
